# Plonk Implementation(Python Version)

## Problem Definition

We take **Square-Fibonacci** as an example to demonstrate the process of proof generation

Defination of Square-Fibonacci Problem:
- Let $f_0 = 1, f_1 = 1$
- For $i \ge 2$, define $f_i:=(f_{i-2})^2+(f_{i-1})^2 \ mod \ q$
    - $q$ is a large prime integer, used to bound the size of each element, so that it can be represented by some predetermined number of bits.

Let $n$ be some very large integer. For convenience, we assume $n$ is a power of 2

Let $k$ be the $n^{th}$ Square-Fibonacci number

**Our goal**: generate an efficiently-verifiable proof $\pi$ showing that indeed $k$ is the $n^{th}$ Square-Fibonacci number(i.e. $f_n = k$)

## Phases of Proof Generation
The Plonk-based proof generation consists of 3 steps:
1. Filling in the trace table
2. Committing to the trace table
3. Proving the trace table's correctness

In [16]:
from sage import *

# some basic statement
q = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
F_q = GF(q)
n = 8
k = 317754178345286893212434
f_0 = F_q(1)
f_1 = F_q(1)

## Step 1: Filling in the Trace Table
The trace table is a 2-dimensional matrix where 'witness' or 'trace' is written down, that is (n rows * 5 cols)
- 5 columns:
    - $A, B, C$: represent witness data / private input, each row lists 3 sequential Sequare-Fibonacci numbers
        - e.g. the $i^{th}$ row $(f_i, f_{i+1}, f_{i+2})$ is a witness for $(i+2)^{th}$ Sequare-Fibonacci number
    - $S$: represents selector column, indicating a certain mathmatical relation should hold over the element of the row.
        - $1$ represents the first 3 elements of the row $(a, b, c)$ must satisfy $c = a^2 + b^2 \ mod \ q$
        - $0$ represents the condition does not need to be satisfied.
    - $P$: represents public inputs, inputs to the circuit that are public known. 
        - e.g. the first two values of the sequence $f_0, f_1$ and $k$ as the value to be proved
- n rows: left a blank row, so that the height of the table becomes $n$, an even power of 2
    - $1^{st}$ row: $f_0, f_1, f_2, 1, f_0$
    - $2^{nd}$ row: $f_1, f_2, f_3, 1, f_1$
    - $3^{rd}$ row: $f_2, f_3, f_4, 1, k$
    - ...
    - $(n-2)^{th}$ row: $f_{n-2}, f_{n-1}, f_n, 1, "" $
    - $(n-1)^{th}$ row: $"", "", "", 0, ""$

Next step is to fill in the trace table: either copy or compute over $F_p$

In [17]:
# generate witness/fill in the trace table
def witness_generation(f_0, f_1, k, n):
    
    trace_table = []
    
    # init col A, B, C, S
    f_a = f_0
    f_b = f_1
    f_c = f_b
    S = F_q(1)
    
    for i in range(n-1):
        f_a = f_b
        f_b = f_c
        f_c = f_a**2 + f_b**2
        trace_table.append([f_a, f_b, f_c, S,-1])
        
    # add a blank row to get n row
    S = 0
    trace_table.append([-1,-1,-1, S,-1])
    
    # add public parameters
    trace_table[0][4] = f_0
    trace_table[1][4] = f_1
    trace_table[2][4] = k
    
    return trace_table

trace_table = witness_generation(f_0, f_1, k, n)
print(f"trace table is {trace_table}")

trace table is [[1, 1, 2, 1, 1], [1, 2, 5, 1, 1], [2, 5, 29, 1, 317754178345286893212434], [5, 29, 866, 1, -1], [29, 866, 750797, 1, -1], [866, 750797, 563696885165, 1, -1], [750797, 563696885165, 317754178345286893212434, 1, -1], [-1, -1, -1, 0, -1]]


## Step 2: Commit to the Trace Table

### interpret the trace table columns as polynomials

Each column can be considered as a length-$n$ vector of finite field elements $\rightarrow$ this vector can be regarded as the evaluation form of a polynomial $A(x)$ with degree $(n-1)$: the $i^{th}$ element of $A$ corresponds to the evaluation $A(\omega^i)$, where $\omega \in F_q$ is **$n^{th}$ root of unity** and has order $n$

In [18]:
# find a subgroup whose order = n
# def find_nth_root_of_unity(F_q, n):
    
#     while True:
#         w = F_q.random_element()
#         if w.multiplicative_order() == n:
#             break
#         else:
#             print(f'w = {w}, whose order is {w.multiplicative_order()}, not statified')
#     return w

# w = find_nth_root_of_unity(F_q, n)
# print(w)

# find a group: {1,2,4,7,8,11,13,14}
w_list = [1,2,4,7,8,11,13,14]

# interpreting the trace table columns as polynomials, and represent the polynomials as evaluation form
evaluation_form = []
for row in range(len(trace_table)):
    evaluation_form_row = []
    for col in range(len(trace_table[0])):
        val = trace_table[row][col]
        evaluation_form_row.append((w_list[row], val))
    evaluation_form.append(evaluation_form_row)
print(evaluation_form)

[[(1, 1), (1, 1), (1, 2), (1, 1), (1, 1)], [(2, 1), (2, 2), (2, 5), (2, 1), (2, 1)], [(4, 2), (4, 5), (4, 29), (4, 1), (4, 317754178345286893212434)], [(7, 5), (7, 29), (7, 866), (7, 1), (7, -1)], [(8, 29), (8, 866), (8, 750797), (8, 1), (8, -1)], [(11, 866), (11, 750797), (11, 563696885165), (11, 1), (11, -1)], [(13, 750797), (13, 563696885165), (13, 317754178345286893212434), (13, 1), (13, -1)], [(14, -1), (14, -1), (14, -1), (14, 0), (14, -1)]]


### Commit to column polynomials
Now that we have known how to interpret the columns as polynomials, we can commit to each of them using a polynomial commitment scheme.

In [19]:
poly_list = []
for col in range(len(trace_table[0])):
    poly_point = []
    for i in range(n):
        poly_point.append(evaluation_form[i][col])
    poly = F_p_x.lagrange_polynomial(poly_point)
    poly_list.append(poly)
print(poly_list)

[30088*x^7 + 50816*x^6 + 98415*x^5 + 9218*x^4 + 84728*x^3 + 24705*x^2 + 69725*x + 93302, 69404*x^7 + 63763*x^6 + 37445*x^5 + 10435*x^4 + 83243*x^3 + 113184*x^2 + 26670*x + 56853, 7192*x^7 + 49562*x^6 + 104482*x^5 + 73122*x^4 + 103274*x^3 + 70563*x^2 + 29003*x + 23800, 69544*x^7 + 27948*x^6 + 57234*x^5 + 18385*x^4 + 35016*x^3 + 13295*x^2 + 95299*x + 29027, 38854*x^7 + 14495*x^6 + 17606*x^5 + 6714*x^4 + 62489*x^3 + 107799*x^2 + 5728*x + 92063]


### Computing KZG commitments

In [20]:
com_list = []
for poly in poly_list:
    com = poly_commitment(pk, g1, poly)
    com_list.append(com)
print(com_list)

[<builtins.G1Point object at 0x112a4f050>, <builtins.G1Point object at 0x112a4ead0>, <builtins.G1Point object at 0x112a4ec30>, <builtins.G1Point object at 0x112a4e970>, <builtins.G1Point object at 0x112a4f100>]


## Step 3: Proving the Trace Table's Correctness
### Define the constraints of the trace table
In order to ensure the original trace table to be valid, we should have the following constraints:
- Square-Fibonaci constraints:
    - each selected row $i$'s first three elements is $(a,b,c)$ must satisfy $c_i = a_i^2 + b_i^2 \ mod \ q$
- Wiring constraints:
    - for consecutive rows with value $[a_i, b_i, c_i]$ and $[a_{i+1}, b_{i+1}, c_{i+1}]$, we require $a_{i+1} = b_i$ and $b_{i+1} = c_i$
- Public input constraints:
    - the first row must start with the first two Square-Fibonacci numbers: $a_0 = p_0, b_0 = p_1$ 
    - the $n^{th}$ Square-Fibonacci must match the claimed result value: $c_{n-2} = p_2$

The above constraints can be represented by one or more relations between the column polynomials. For example, Square-Fibonaci constraints can be expressed as $S(x)·(A(x)^2 + B(x)^2 - C(x))=0, for\ all\ x\in \{w^0, w^1, ..., w^{n-1}\}$. For shorten, we will label left-hand side $\phi_0(x):=S(x)·(A(x)^2 + B(x)^2 - C(x))$  All our constraints can be expressed as $\phi_i(x) = 0,\ for\ all\ x\in \{w^0, w^1, ..., w^{n-1}\}$

In [23]:
constraints_poly = []
# Square-Fibonaci constraints
Fi_0 = poly_list[3]*(poly_list[0]**2 + poly_list[1]**2 - poly_list[2])

# wiring constraints


Fi = Fi_0

### Combine constraints

- **Naive way to proof the contraints**

    In general, we have $m$ constraint polynomials $\phi_0(x), \phi_1(x), ..., \phi_{m-1}(x)$. Sample a random field element $\gamma \in F_q$, and then take a random linear combination of the individual constraints:$$\phi(x) := \gamma^0·\phi_0(x) + \gamma^1·\phi_1(x)+...+\gamma^{m-1}·\phi_{m-1}(x)$$
    and we need the constraints satisfied at every row, that is $\phi(\omega^i) = 0 \ for \ all\ 0\le i <n$, in this case we need $n$ evaluation proofs


- **Using quotient polynomial**

    we can prove such constraint $\phi(x)$ using only one evaluation proof:
    - quotient polynomial:
        $$
        \begin{aligned} 
        \phi(\omega^i) = 0 \ for \ all\ 0\le i <n\ &{\Leftrightarrow}\ (x-\omega^i)|\phi(x)\ for \ all\ 0\le i<n\\
        &{\Leftrightarrow}\ \prod^{n-1}_{i=0}(x-\omega^i)|\phi(x)\ for \ all\ 0\le i <n\\
        &{\Leftrightarrow}\ (x^n-1)|\phi(x)\\
        &{\Leftrightarrow}\ \exists Q(x)\ s.t.\phi(x)=Q(x)·(x^n-1)
        \end{aligned}
        $$
     now we just need to prove there exists a polynomial $Q(x)$
    - compute the quotient polynomial:
        $$
        Q(x):= {\phi(x) \over {x^n-1}} = {{\gamma^0·\phi_0(x) + \gamma^1·\phi_1(x)+...+\gamma^{m-1}·\phi_{m-1}(x)} \over {x^n-1}} 
        $$
      degree of $Q(x)$ is $2n-3$, so we need at least $2n-2$ evaluation points to represent it
      
      we make it a round number and use $2n$ evaluation points, our previous evaluation domain do not work anymore, because the order of $\omega$ is only $n$. We therefore need to pick some other element $\beta \in F_q$ with order $2n$. Then we evaluate $Q(x)$ over the evaluation domain $\{\beta^0, \beta^1,...,\beta^{2n-1}\}$
      

### Committing to the quotient polynomial
Now we have get the evaluation form of $Q(x)$, we can compute its commitment. Note, degree of $Q(x)$ is larger than the column polynomials, so it requires a larger KZG setup

(in practice, we can use a trick here: split $Q(x)$ into 2 smaller polynomials with each one's degree $<n$, and commit them seperately: $Q(x) = Q_{lo}(x) + x^n·Q_{hi}(x)$ )

### Proving the quotient polynomial's existence
Now we have committed all column polynomials from the trace table and have also committed to the quotient polynomial. Now the prover needs to demonstrate the quotient polynimial really exist and it was computed correctly.

This can be achieve through the following steps:
1. sample a random point $\alpha \in F_q$
2. generate and output KZG proofs for all column polynimials and the quotient polynomials at point $\alpha$
    $$
    Q(\alpha):= {\phi(\alpha) \over {\alpha^n-1}} = {{\gamma^0·\phi_0(\alpha) + \gamma^1·\phi_1(\alpha)+...+\gamma^{m-1}·\phi_{m-1}(\alpha)} \over {\alpha^n-1}} 
    $$
    $\alpha$ is sampled at random, so the property holds at $\alpha$, then it holds everywhere

### Verifying the quotient polynomial
the verifier need to check two things:
1. each evaluation proof is correct
2. the quotient polynomial holds at point $\alpha$

# Appendix

## KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

## Step 0: Curve Preparation

Curve we used is BLS12-381
- Library we used: BLS21-381 curve implemented by Arkwork, using Python
- Lib doc: https://pypi.org/project/py-arkworks-bls12381/

In [6]:
!pip install py-arkworks-bls12381


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Step 1: Setup

The first step is an one-time trusted setup and once it has done once, the following steps can be done repeatedly
1. Let $G_1$ and $G_2$ be pairing-friendly elliptic curve groups
2. Let $g_1$ be a generator of $G_1$ and $g_2$ be a generator of $G_2$
3. Let $l$ be the maximum degree of the polymonials we want to commit to ($l < p$)
4. Pick a random field element as secret parameter $\tau \in F_p$(usually done by MPC, to simplify, we just randomly choose one here)
5. Compute pp(public parameters)$$pk = (g_1, g_1^\tau, g_1^{\tau^2},...,g_1^{\tau^l}), vk = g_2^\tau$$ and release it publicly
6. Discard secret parameter $\tau$ once the setup ceremony is done so that nobody can figure out its value

In [7]:
from py_arkworks_bls12381 import G1Point, G2Point, Scalar
from sage import *

# 1. Let G be a pairing-friendly elliptic curve group, and G1 is the G

# 2. Let g be a generator of G
g1 = G1Point()
g2 = G2Point()

# 3. Let l be the maximum degree of the polymonials, which is 12(by the definition of the curve)
l = 12

# 4. Pick a random field element as secret parameter t
# ! cannot use real p, when exceed 2^127, scalar will warning "overflow"
# p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
# F_p = GF(p)
# t = F_p.random_element()
p = 115249
F_p = GF(p)
t = F_p.random_element()
print(f'secret parameter 𝜏 is {t}')

# 5. Compute pp(public parameters)
def compute_public_parameters(g1, g2, t, l):
    
    pk = []
    accumulated = Scalar(1)
    t_scalar = Scalar(t)
    for i in range(l + 1):
        pk.append(g1 * accumulated)
        accumulated = accumulated * t_scalar
    vk = g2 * Scalar(t)
    return pk, vk

pk, vk = compute_public_parameters(g1, g2, t, l)
print(f'pk: {pk}, vk: {vk}')

secret parameter 𝜏 is 5747
pk: [<builtins.G1Point object at 0x112a4e130>, <builtins.G1Point object at 0x112a4e080>, <builtins.G1Point object at 0x112a4e1e0>, <builtins.G1Point object at 0x112a4e290>, <builtins.G1Point object at 0x112a4e340>, <builtins.G1Point object at 0x112a4e3f0>, <builtins.G1Point object at 0x112a4e4a0>, <builtins.G1Point object at 0x112a4e550>, <builtins.G1Point object at 0x112a4e600>, <builtins.G1Point object at 0x112a4e6b0>, <builtins.G1Point object at 0x112a4e760>, <builtins.G1Point object at 0x112a4e810>, <builtins.G1Point object at 0x112a4e8c0>], vk: a201e718afed0367e547d76e245057cd1b39e171328c343a075abedd6ecb74ddc54d46a7b1c311a67d5253373bca6f5f1414bf06438372210570daadcda96fc570de53124e6a57e9b337eeaa378cddb6e72892ae64290c80cae5870b7a0ec003


## Step 2: Commit to Polynomials
In reality, we arithmetize circuits and use Plonkish to get polynomials in this step
1. Given a polynomial $\Phi(x) = \sum_{i=0}^l \phi_i x^i$
2. Compute and output commitment $c = g^{\Phi(\tau)}$
   - Wait! $\tau$ has already been discard right? How can committer compute $\tau$?
   - Although he cannot compute $\Phi(\tau)$ directly, he can use public parameters to help with it: 
$$\prod_{i=0}^l(g^{\tau^i})^{\phi_i} = g^{\sum_{i=0}^l \phi_i \tau^i} = g^{\Phi(\tau)}$$

In [8]:
# assume we have done plonk and get our polynomial Φ(𝑥)=4x^2+5x+3 and some point (0,3),(1,12),(3,54) on the poly
# poly_coefs = [3, 5, 4]
poly_point = [(0,3),(1,12),(3,54)]

In [9]:
# create a polynomial ring F_p(x)
F_p_x.<x> = PolynomialRing(F_p)
F_p_x

Univariate Polynomial Ring in x over Finite Field of size 115249

In [10]:
poly = F_p_x.lagrange_polynomial(poly_point)
poly

4*x^2 + 5*x + 3

In [11]:
def poly_commitment(pk, g1, poly):

    poly_coefs = poly.coefficients()
    com = G1Point.identity()
    
    for i in range(len(poly_coefs)):
        com = pk[i] * Scalar(poly_coefs[i]) + com
    return com
        
com = poly_commitment(pk, g1, poly)
print(f'commitment of polynomial is {com}')

commitment of polynomial is b4b882fbc091071b7d7f245bb56de3148ae627aa9db1fdcfa38ced8bb47222890940ffa078f150c74fc40d535d04db97


## Step 3: Prove an Evaluation
In this period, the Verifier will ask Prover to 'OPEN' the commitment $c$ to a random specific point $a \in F_p$, in other word, Prover have to evaluation $\Phi(x)$ and commit the result in the form of opening triplet $OT = (a, b, \pi)$
1. Given an evaluation $\Phi(a) = b$
2. Compute and output proof of the evaluation $\pi = g^{q(\tau)}$, where $q(x) := \frac{\Phi(x)-b}{x-a}$
    - $q(x)$ is quotient polynomial: if $\Phi(a) = b$, that means $a$ is a root of $\Phi(x) - b$
    - so $\Phi(x) - b$ can be expressed as $\Phi(x) - b = q(x)(x-a)$, $q(x)$ is a polynomial
    - on the other hand, $q(x)$ exists if and only if $\Phi(a) = b$, so the existence of this quotient polynomial therefore serves as a proof of the evaluation

In [12]:
# Verifier first choose the random point a
# a = F_p.random_element()
# b = poly(a)
a = 1
b = poly(a)

print(f'random element we chosen is {a}, the evaluation b is {b}')

# compute q(x)
q_poly = ((poly-b)/(x-a)).numerator()
print(f'quotient polynomial is {q_poly}')

# compute proof of the evaluation pi
pi = poly_commitment(pk, g1,  q_poly)
print(f'proof of the evaluation pi is {pi}')

random element we chosen is 1, the evaluation b is 12
quotient polynomial is 4*x + 9
proof of the evaluation pi is 8a2569d40ee9d38b3ac95f53bcf0c896660f7120f6f2c3e9f7a56f6e6df2bd7b5fad2c8fed84388515d1ba1ae4a72d6a


## Step 4: Verify an Evaluation Proof
1. Given a commitment $c = g^{\Phi(\tau)}$, and an evaluation $\Phi(a) = b$, and a proof $\pi = g^{q(\tau)}$
2. Verify that $e(\frac{c}{g^b}, g) = e(\pi,\frac{g^\tau}{g^a})$, where $e$ is a non-trivial bilinear mapping
    - the purpose of verification: $\Phi(x) - b = q(x)(x-a)$, checking this equality holds for $x = \tau$
    - according to the definition of bilinear mapping, it is equivalent to: $e(g_1, g_2)^{\Phi(\tau) - b} = e(g_1, g_2)^{q(\tau)(\tau-a)}$, that is $e(g_1^{\Phi(\tau) - b}, g_2) = e(g_1^{q(\tau)}, g_2^{\tau-a})$
    - that is $e(com-g_1^b, g_2) = e(\pi, vk - g_2^a)$

In [13]:
from py_arkworks_bls12381 import GT


# now it is time to do the varification
assert GT.pairing(com - g1*Scalar(b), g2) == GT.pairing(pi, vk - g2*Scalar(a))

In [14]:
GT.pairing(com - g1*Scalar(b), g2)

In [15]:
GT.pairing(pi, vk - g2*Scalar(a))